In [1]:
#import necessary libraries 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
# get the url text and parse it to extract the table colunn values
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url = requests.get(url).text

soup = BeautifulSoup(url, 'html.parser')
table = soup.find('table', class_='sortable')

#make a list of items to store the extraction results
data = []
for tr in table.find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue
    zip_code, borough, neighbuorhood = [td.text.strip() for td in tds[:3]]
    data.append([zip_code, borough, neighbuorhood])


In [3]:
# transfer the results into a df and assign col. labels, drop 'Not assigned' values of the Borough column

df = pd.DataFrame(data, columns = ['Postcode', 'Borough', 'Neighbourhood'])
df = df[(df['Borough'] != 'Not assigned') ].reset_index(drop = True)


s = df.groupby(['Postcode','Borough'], 
               as_index = False)['Neighbourhood'].apply(lambda x: '{}'.format(', '.join(x))).replace(lambda x,y: 
                                                                                                    x.replace('Not assigned'),)
df = s.to_frame().sort_values(by = 'Postcode').reset_index(drop=False)
df.columns = ['Postcode', 'Borough', 'Neighbourhood']


In [4]:
# define a function to deal with 'Not assigned' values of the Neighbourhood column
def fill_missing(input):
    bor, neigh = input
    if neigh == 'Not assigned':
        neigh = bor
    else:
        neigh
    return neigh

fill_missing(df.iloc[85,1:])

"Queen's Park"

In [5]:
df['Neighbourhood'] = df[['Borough','Neighbourhood']].apply(fill_missing, axis = 1)

In [6]:
df[df['Postcode'] == 'M7A']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [7]:
#shape of the final df.
df.shape

(103, 3)